In [3]:
df.shape

(50796, 26)

In [2]:
df.head()

,会员账号,币种,日期,上级代理,vip,存款金额,提款金额,平台坐庄-投注金额,平台坐庄-下注金额,平台坐庄-派彩金额,...,返水金额,投注次数,有效投注金额,会员属性,二级部门,活跃人数,一级部门,不结算人数,月份,22
0,mac0066,CNY,2023/10/5,btydl00033,2,0.0,0,0.0,0.0,0.0000,...,2.13,0,0.0,正常,btydl0099,0,袋鼠直播,0,10,btydl00033
1,Itobspor20,CNY,2023/10/5,btyseo3636,2,0.0,0,36.0,60.0,39.1200,...,0.18,5,36.0,正常,鲲鹏seo,1,鲲鹏seo,0,10,btyseo3636
2,wd210802,CNY,2023/10/5,btydl00203,1,0.0,0,60.0,60.0,55.2000,...,0.12,2,60.0,正常,btydl0020,1,小Peter,0,10,btydl00203
3,liuning08,CNY,2023/10/5,btyseo3000,3,0.0,0,39.3,39.3,26.3964,...,12.91,6,39.3,正常,鲲鹏seo,1,鲲鹏seo,0,10,btyseo3000
4,jun125347,CNY,2023/10/5,btyseo8003,4,0.0,0,0.0,0.0,0.0000,...,0.04,0,0.0,正常,鲲鹏seo,0,鲲鹏seo,0,10,btyseo8003


In [1]:
import pandas as pd
df = pd.read_clipboard()
df.shape

(50796, 26)

In [15]:
# SEO环境： schedule
import os
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = -1
pages_user = 150
pages_fircharge = 60
pages_fircharge_two = 100
pages_trade = 150

url_trade ='http://fundmng.bsportsadmin.com/api/manage/data/balance/record/list'
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'

# 会员流失统计表
huiyuan = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1011\流失会员详情.csv',encoding='gbk')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO提单数据\1011\代理线.xlsx')
# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


# 采集会员列表和会员首存记录
session = requests.session()

# 采集昨日首存报表
dic_fir = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
# 昨天开始时间戳
yesterday = datetime.date.today() + datetime.timedelta(days=day)
yesterday_start_time = int(time.mktime(time.strptime(str(yesterday), '%Y-%m-%d')))
# 昨天结束时间戳
yesterday_end_time = int(time.mktime(time.strptime(str(datetime.date.today()), '%Y-%m-%d'))) - 1

#---------------------token----------------------
# 采取token
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'75',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=67c8b1bd1b434f898ed8570a860355b8; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100112',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/report-first-recharge',
    'Sign':'ca83944852acc68fe114cbc65f1e1d22',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1692092554000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
for page in range(1,pages_fircharge+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': yesterday_end_time*1000+999
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir['会员名'].append(i['userName'])
        dic_fir['所属代理'].append(i['parentName'])
        dic_fir['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir['交易类型'].append(i['tradeType'])
        dic_fir['币种'].append(i['coinCode'])
        dic_fir['金额'].append(i['amount'])
firChargeUser = pd.DataFrame(dic_fir)
print('会员首存行列：',firChargeUser.shape)

# 2.采集昨天和今天的首存报表
dic_fir_two = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
for page in range(1,pages_fircharge_two+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': int(time.time())*1000
    }
    response = session.post(url_fircharge,headers=header,data=data)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir_two['会员名'].append(i['userName'])
        dic_fir_two['所属代理'].append(i['parentName'])
        dic_fir_two['注册时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['createTime']//1000)))
        dic_fir_two['交易时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['regTime']//1000)))
        dic_fir_two['交易类型'].append(i['tradeType'])
        dic_fir_two['币种'].append(i['coinCode'])
        dic_fir_two['金额'].append(i['amount'])
charge_two  = pd.DataFrame(dic_fir_two)
print('昨天+今天的会员首存行列：',charge_two.shape)

# 采集会员列表
dic_user = dict({'会员账号':[], '姓名':[],'代理':[],'玩家层级':[], '注册时间':[], '首存时间':[],'备注':[]})
for page in range(1,pages_user+1):
    data2 = {
        'page':page,
        'size':20,
        'userVip':'0,1,2,3,4,5,6,7,8,9,10,11',
        'status':'0,1,2,4',
        'sortType':'3',
        'sortStr':'descend',
        'searchType':'1',
        'channelId':'34',
        'registeredStartDate':yesterday_start_time*1000,
        'registeredEndDate':yesterday_end_time*1000+999,
    }
    response2 = session.post(url_user,headers=header,data=data2)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_user['会员账号'].append(i['username'])
        if i['reallyName'] !='':
            dic_user['姓名'].append(i['reallyName'])
        else:
            dic_user['姓名'].append('-')

        dic_user['代理'].append(i['parentName'])
        dic_user['玩家层级'].append(i['levelName'])
        dic_user['注册时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['registerDate']//1000)))
        dic_user['首存时间'].append(time.strftime('%Y/%m/%d',time.localtime(i['firstRechargeDate']//1000)))
        dic_user['备注'].append(i['remark'])
user = pd.DataFrame(dic_user)
print('用户列表行列:',user.shape)
# 删除测试账号
user = user[~user['会员账号'].str.contains('test')&~user['会员账号'].str.contains('ceshi')&~user['姓名'].str.contains('测试') \
            &~user['姓名'].str.contains('ceshi')&~user['代理'].str.contains('测试')&~user['代理'].str.contains('ceshi') \
            &~user['备注'].str.contains('测试')&~user['备注'].str.contains('试玩')&~user['备注'].str.contains('晒单') \
            &~user['玩家层级'].str.contains('测试')]
print('去重后：',user.shape)

shuju = pd.DataFrame({'人员':['paddy', 'tony', 'max', 'martin', 'zed', 'hugo', 'aber', 'dk', 'ben']})
shuju['人员2']=shuju['人员']
shuju.set_index('人员2',inplace=True)
shuju.sort_index(inplace=True)
# 注册数

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
result = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
result.rename(columns={'seo变化数据团队':'注册数'},inplace=True)

# 绑卡
merge_user_dropna = merge_user.loc[~merge_user['seo变化数据团队'].isna()&(merge_user['姓名']!='-'),]
bangka = merge_user_dropna.groupby('seo变化数据团队').agg({'会员账号':len})
bangka.rename(columns={'会员账号':'绑卡数'},inplace=True)

# join
result2 = result.join(bangka).fillna(0)
result2['绑卡率'] = result2['绑卡数']/result2['注册数']


# 采集交易明细 trade
# 新token
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'103',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=fc66d42ccb0f4ca58b184fbe5498158c; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100104',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/transaction-for-user',
    'Sign':'1abc87008c66bc0b4d3d7a3870442b16',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1697016616000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}

dic_trade = dict({'账户名':[], '所属代理':[]})
for page in range(1,pages_trade+1):
    data2 = {
        'page':page,
        'size':500,
        'reportType':'0',
        'userType':'0',
        'dateType':'0',
        'startDate':yesterday_start_time*1000,
        'endDate':yesterday_end_time*1000+999,
        'type':'1'
    }
    response2 = session.post(url_trade,headers=header,data=data2)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_trade['账户名'].append(i['username'])
        dic_trade['所属代理'].append(i['parentName'])

trade = pd.DataFrame(dic_trade)
print('trade:',trade.shape)
# 未拉过单
huiyuan2 = huiyuan.loc[huiyuan['首存时间']==0,]
merge_huiyuan = huiyuan2.merge(daili,left_on = '代理',right_on='代理线')
# print(merge_huiyuan.shape)
# merge_huiyuan2 = merge_huiyuan.loc[~merge_huiyuan['准点激活团队'].isna(),]
# print('处理后：',merge_huiyuan2.shape)

mingdan= set(list(merge_huiyuan['会员账号'].values)+list(charge_two['会员名'].values))
# 删除交易测试账号
trade2 = trade[~trade['账户名'].str.contains('test')&~trade['账户名'].str.contains('ceshi')&~trade['所属代理'].str.contains('测试')&~trade['所属代理'].str.contains('ceshi')]
trade_daili= trade2.merge(daili,left_on='所属代理',right_on='代理线')
trade_daili.head()

会员首存行列： (336, 7)
昨天+今天的会员首存行列： (435, 7)
用户列表行列: (935, 7)
去重后： (841, 7)
trade: (36856, 2)


,账户名,所属代理,代理线,主线账号,seo变化数据团队,准点激活团队
0,WXQ888888,btyseo8003,btyseo8003,btyseo8003,aber,SEO
1,l13190342180,btyseo8003,btyseo8003,btyseo8003,aber,SEO
2,jj7289,btyseo8003,btyseo8003,btyseo8003,aber,SEO
3,FV88888888,btyseo8003,btyseo8003,btyseo8003,aber,SEO
4,WXQ888888,btyseo8003,btyseo8003,btyseo8003,aber,SEO


In [21]:
tidan = trade_daili.loc[trade_daili['账户名'].isin(mingdan),]
tidan.shape

(388, 6)

In [ ]:
huiyuan.merge(daili,left_on='代理',right_on='代理线')

In [ ]:
tidan = trade_daili.loc[trade_daili['账户名'].isin(mingdan),]
# 去除重复用户
tidan.drop_duplicates('账户名',inplace=True)
tidan_grp = tidan.groupby('seo变化数据团队').agg({'账户名':len}).rename(columns={'账户名':'提单人数'})
shuju = shuju.join(result2).join(tidan_grp)
shuju.fillna(0,inplace=True)
shuju['提单率']=shuju['提单人数']/shuju['绑卡数']

In [2]:
import pandas as pd
df1 = pd.read_csv(r'C:\Users\User\Desktop\9月充值异常记录\充值异常记录20231005130430-246.csv',encoding='gbk')
df2 = pd.read_csv(r'C:\Users\User\Desktop\9月充值异常记录\充值异常记录20231005130937-249.csv',encoding='gbk')
df3 = pd.read_csv(r'C:\Users\User\Desktop\9月充值异常记录\充值异常记录20231005131427-86.csv',encoding='gbk')
df =df1.append(df2).append(df3)
df.shape

C:\Users\User\AppData\Local\Temp\ipykernel_6672\3949895063.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df1.append(df2).append(df3)


(1158941, 11)

In [24]:
merge_huiyuan.shape

(31428, 17)

In [14]:
result = df.loc[df['账户名']=='Cds123456789',].groupby('异常原因').agg({'账户名':len})
result.rename(columns={'账户名':'Cds123456789异常原因次数'},inplace=True)
result.sort_values('Cds123456789异常原因次数',ascending=False)

,Cds123456789异常原因次数
异常原因,
"[质远支付_561302 超出收款金额范围, 超级支付V2_CJZF8888 超出收款金额范围]",7215
[超级支付V2_CJZF8888 超出收款金额范围],1225
[码上支付_M7GXJSR4AG 超出收款金额范围],1041
"[超级支付V2_CJZF8888 超出收款金额范围, 质远支付_561302 超出收款金额范围]",837
"没有符合条件的收款银行卡,请检查风控以及状态",421
"[质远支付_561302 超出收款金额范围, 码上支付_M7GXJSR4AG 超出收款金额范围]",334
"没有符合条件的收款银行卡,当日交易次数以及交易总额超限",169
"[鑫鑫支付V2_75021365 商户余额超过收款限额, MGO支付_PF0037 超出收款金额范围]",49
"[质远支付_561302 超出收款金额范围, 超级支付V2_CJZF8888 超出收款金额范围, 全通支付_d27dcad1-9b9c-41f4-aab3-360586d3f947 超出收款金额范围]",41


In [4]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

user = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员列表导出.csv',encoding='gbk')
firChargeUser = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员首存报表.csv',encoding='gbk')
data = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\data\今日数据.xlsx')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\SEO每日模板-每日更新.xlsx','代理总表')

shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=-3)).strftime('%Y/%m/%d'),
                      '发送IP':[i for i in range(10)],'接受IP':[i for i in range(10)]})


shuju.set_index('人员',inplace = True)
shuju.sort_index(inplace=True)
data['IP']=pd.to_numeric(data['IP'],errors='coerce').replace(np.nan,0)
grp=data.groupby('网站名(domain)').agg({'IP':sum})

shuju.loc['Paddy','发送IP']=grp.loc['paddy.com','IP']
shuju.loc['Paddy','接受IP']=grp.loc['paddy.bty','IP']
shuju.loc['Tony','发送IP']=grp.loc['tonyb.com','IP']/2
shuju.loc['Tony','接受IP']=grp.loc['tonyb.com','IP']
shuju.loc['Max','发送IP']=grp.loc['mulu.com','IP']
shuju.loc['Max','接受IP']=grp.loc['max.bty','IP']
shuju.loc['Martin','发送IP']=grp.loc['redquan.com','IP']
shuju.loc['Martin','接受IP']=grp.loc['martin.bty','IP']
shuju.loc['Zed','发送IP']=grp.loc['zed.com','IP']
shuju.loc['Zed','接受IP']=grp.loc['zed.bty','IP']
shuju.loc['Hugo','发送IP']=grp.loc['hugo.com','IP']
# shuju.loc['Hugo','接受IP']=grp.loc['hugo.bty','IP']
shuju.loc['Hugo','接受IP']=0

shuju.loc['Aber','发送IP']=grp.loc['aber.com','IP']/2
shuju.loc['Aber','接受IP']=grp.loc['aber.bty','IP']
shuju.loc['DK','发送IP']=grp.loc['dk.com','IP']/2
shuju.loc['DK','接受IP']=grp.loc['dk.bty','IP']
shuju.loc['Ben','发送IP']=grp.loc['ben.com','IP']/2
shuju.loc['Ben','接受IP']=grp.loc['ben.bty','IP']
shuju.loc['当日汇总','发送IP']=shuju['发送IP'].sum()
shuju.loc['当日汇总','接受IP']=shuju['接受IP'].sum()

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()
shuju

C:\ProgramData\anaconda3\envs\schedule\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,IP
网站名(domain),
aber.bty,562.0
aber.com,14650.0
aber.zb,0.0
aber.zz,0.0
alen.bty,0.0
ben.bty,298.0
ben.com,5130.0
ben.zb,4.0
ben.zz,0.0


,日期,人员2,发送IP,接受IP
人员,,,,
Aber,2023-08-07,aber,7325,562
Ben,2023-08-07,ben,2565,298
DK,2023-08-07,dk,1525,2160
Hugo,2023-08-07,hugo,103,0
Martin,2023-08-07,martin,3445,1072
Max,2023-08-07,max,273,225
Paddy,2023-08-07,paddy,0,51
Tony,2023-08-07,tony,568,1136
Zed,2023-08-07,zed,4853,329


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [1]:
import os
os.__file__

'C:\\ProgramData\\anaconda3\\Lib\\os.py'

In [2]:
import pandas as pd
df = pd.read_clipboard()

In [3]:
df.head()

,会员名,所属代理,注册时间,交易时间,交易类型,币种,金额
0,ngthduyen9999,btyscnb0007,2023/8/7 23:22,2023/8/7 23:43,存款,VND,11000000.0
1,Phamvquyen91,vnzb04,2023/8/7 23:27,2023/8/7 23:39,存款,VND,100000.0
2,Angunb666,zh0325,2023/8/7 23:03,2023/8/7 23:34,存款,CNY,200.0
3,Guong268,gfas001,2023/5/13 23:45,2023/8/7 23:30,存款,VND,4000000.0
4,zxcvbnm19064,btyseo7008,2022/7/30 23:35,2023/8/7 23:31,存款,CNY,100.0


In [6]:
import os
os.__file__

'C:\\ProgramData\\anaconda3\\Lib\\os.py'